In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import xgboost as xgb
from time import time

In [5]:
path = '/Users/Vishy/Files/AVDatahack/LOM/Data/'
train = pd.read_csv(path+'train.csv')
test = pd.read_csv(path+'test.csv')
camp = pd.read_csv(path+'campaign_data.csv')
print(train.shape, test.shape, camp.shape)

(1023191, 6) (773858, 4) (52, 9)


In [6]:
train.head()

,id,user_id,campaign_id,send_date,is_open,is_click
0,42_14051,14051,42,01-09-2017 19:55,0,0
1,52_134438,134438,52,02-11-2017 12:53,0,0
2,33_181789,181789,33,24-07-2017 15:15,0,0
3,44_231448,231448,44,05-09-2017 11:36,0,0
4,29_185580,185580,29,01-07-2017 18:01,0,0


In [7]:
test.head()

,id,campaign_id,user_id,send_date
0,63_122715,63,122715,01-02-2018 22:35
1,56_76206,56,76206,02-01-2018 08:15
2,57_96189,57,96189,05-01-2018 18:25
3,56_166917,56,166917,02-01-2018 08:15
4,56_172838,56,172838,02-01-2018 08:12


In [8]:
camp.head()

,campaign_id,communication_type,total_links,no_of_internal_links,no_of_images,no_of_sections,email_body,subject,email_url
0,29,Newsletter,67,61,12,3,"Dear AVians,\r\n \r\nWe are shaping up a super...",Sneak Peek: A look at the emerging data scienc...,http://r.newsletters.analyticsvidhya.com/7um44...
1,30,Upcoming Events,18,14,7,1,"Dear AVians,\r\n \r\nAre your eager to know wh...",[July] Data Science Expert Meetups & Competiti...,http://r.newsletters.analyticsvidhya.com/7up0e...
2,31,Conference,15,13,5,1,Early Bird Pricing Till August 07  Save upto ...,Last chance to convince your boss before the E...,http://r.newsletters.analyticsvidhya.com/7usym...
3,32,Conference,24,19,7,1,\r\n \r\nHi ?\r\n \r\nBefore I dive into why y...,A.I. & Machine Learning: 5 reasons why you sho...,http://r.newsletters.analyticsvidhya.com/7uthl...
4,33,Others,7,3,1,1,Fireside Chat with DJ Patil - the master is he...,"[Delhi NCR] Fireside Chat with DJ Patil, Forme...",http://r.newsletters.analyticsvidhya.com/7uvlg...


In [11]:
camp.total_links.nunique()

31

In [12]:
train = pd.merge(train, camp, on='campaign_id')
test = pd.merge(test, camp, on='campaign_id')
print(train.shape, test.shape)

(1023191, 14) (773858, 12)


In [13]:
train.columns

Index(['id', 'user_id', 'campaign_id', 'send_date', 'is_open', 'is_click',
       'communication_type', 'total_links', 'no_of_internal_links',
       'no_of_images', 'no_of_sections', 'email_body', 'subject', 'email_url'],
      dtype='object')

In [14]:
test.columns

Index(['id', 'campaign_id', 'user_id', 'send_date', 'communication_type',
       'total_links', 'no_of_internal_links', 'no_of_images', 'no_of_sections',
       'email_body', 'subject', 'email_url'],
      dtype='object')

In [15]:
# Process date variable
# 01-02-2018 22:35
# 2017-08-21 00:00:01
train['send_date'] = pd.to_datetime(train['send_date'])
test['send_date'] = pd.to_datetime(test['send_date'])

In [16]:
# Feature Enginerring to Create New variables for Day of Week and Day part
#train
train['Weekday'] = (train['send_date'].dt.dayofweek).astype(int)
train['Dayhour'] = (train.send_date.apply(lambda x: x.hour)).astype(int)
train['Month'] = pd.DatetimeIndex(train['send_date']).month

#test
test['Weekday'] = (test['send_date'].dt.dayofweek).astype(int)
test['Dayhour'] = (test.send_date.apply(lambda x: x.hour)).astype(int)
test['Month'] = pd.DatetimeIndex(test['send_date']).month

In [17]:
# To remove Ordinality in both Weekday and DayHour variables convert into Categories
#train
train['Weekday'] = train['Weekday'].map({0:'Mon', 1:'Tue', 2:'Wed', 3:'Thr',4: 'Fri', 5: 'Sat', 6: 'Sun'} ).astype(str)
train['Dayhour'] = train['Dayhour'].map({0:'AM1', 1:'AM1', 2:'AM1', 3:'AM2', 4:'AM2', 5:'AM2', 6:'AM3', 7:'AM3', 8:'AM3',
                                         9:'AM4', 10:'AM4', 11:'AM4', 12:'PM1', 13:'PM1', 14:'PM1', 15:'PM2', 16:'PM2',
                                         17:'PM2', 18:'PM3', 19:'PM3', 20:'PM3', 21:'PM4', 22:'PM4', 23:'PM4'}).astype(str)
train['Month'] = train['Weekday'].map({1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun', 7:'Jul', 8:'Aug', 9:'Sep', 
                                        10:'Oct', 11:'Nov', 12:'Dec'} ).astype(str)

#test
test['Weekday'] = test['Weekday'].map({0:'Mon', 1:'Tue', 2:'Wed', 3:'Thr', 4:'Fri', 5:'Sat', 6:'Sun'}).astype(str)
test['Dayhour'] = test['Dayhour'].map({0:'AM1', 1:'AM1', 2:'AM1', 3:'AM2', 4:'AM2', 5:'AM2', 6:'AM3', 7:'AM3', 8:'AM3',
                                       9:'AM4', 10:'AM4', 11:'AM4', 12:'PM1', 13:'PM1', 14:'PM1', 15:'PM2', 16:'PM2',
                                       17:'PM2', 18:'PM3', 19:'PM3', 20:'PM3', 21:'PM4', 22:'PM4', 23:'PM4'}).astype(str)
test['Month'] = test['Weekday'].map({1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun', 7:'Jul', 8:'Aug', 9:'Sep', 
                                        10:'Oct', 11:'Nov', 12:'Dec'}).astype(str)

In [18]:
# Create final day time variable
train['DayPeriod'] = (train['Month']+train['Weekday']+train['Dayhour']).astype(str)
test['DayPeriod'] = (test['Month']+test['Weekday']+train['Dayhour']).astype(str)
del train['send_date'],train['Weekday'],train['Dayhour'],test['send_date'],test['Weekday'],test['Dayhour'],train['Month'],test['Month']

In [19]:
print (train.campaign_id.value_counts(normalize=True).head())
print ('------------------------------------------------------')
print (train.communication_type.value_counts(normalize=True).head())
print ('------------------------------------------------------')
print (train.user_id.value_counts(normalize=True).head())
print ('------------------------------------------------------')
print (train.DayPeriod.value_counts(normalize=True).head())
print ('------------------------------------------------------')
print (train.is_open.value_counts(normalize=True).head())
print ('------------------------------------------------------')
print (train.is_click.value_counts(normalize=True).head())

54    0.091814
53    0.083495
52    0.080298
49    0.079514
42    0.079411
Name: campaign_id, dtype: float64
------------------------------------------------------
Newsletter         0.440830
Conference         0.325094
Upcoming Events    0.145588
Others             0.049782
Hackathon          0.032511
Name: communication_type, dtype: float64
------------------------------------------------------
183177    0.000020
145022    0.000020
4118      0.000019
216700    0.000019
114180    0.000019
Name: user_id, dtype: float64
------------------------------------------------------
nanSunPM4    0.154950
nanSatPM1    0.119108
nanThrPM3    0.097015
nanThrPM2    0.082237
nanMonPM3    0.079411
Name: DayPeriod, dtype: float64
------------------------------------------------------
0    0.89954
1    0.10046
Name: is_open, dtype: float64
------------------------------------------------------
0    0.987508
1    0.012492
Name: is_click, dtype: float64


In [20]:
test.columns

Index(['id', 'campaign_id', 'user_id', 'communication_type', 'total_links',
       'no_of_internal_links', 'no_of_images', 'no_of_sections', 'email_body',
       'subject', 'email_url', 'DayPeriod'],
      dtype='object')

In [21]:
# prep data for Base Model
target_train = train['is_click']
id_test = test['id']

del train['id'],train['user_id'],train['is_open'],train['is_click'],train['email_body'],train['subject'],train['email_url']
del test['id'],test['user_id'],test['email_body'],test['subject'],test['email_url']

print(train.shape,test.shape)

(1023191, 7) (773858, 7)


In [23]:
train_cats = pd.get_dummies(data=train, columns=['campaign_id','communication_type','DayPeriod'], drop_first=True)
test_cats = pd.get_dummies(data=test, columns=['campaign_id','communication_type','DayPeriod'], drop_first=True)
print (train_cats.shape,test_cats.shape)

(1023191, 54) (773858, 54)


In [24]:
train_cats.head()

,total_links,no_of_internal_links,no_of_images,no_of_sections,campaign_id_30,campaign_id_31,campaign_id_32,campaign_id_33,campaign_id_34,campaign_id_35,...,DayPeriod_nanSunPM4,DayPeriod_nanThrPM1,DayPeriod_nanThrPM2,DayPeriod_nanThrPM3,DayPeriod_nanThrPM4,DayPeriod_nanTueAM4,DayPeriod_nanTuePM3,DayPeriod_nanWedAM4,DayPeriod_nanWedPM1,DayPeriod_nanWedPM2
0,88,79,13,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,88,79,13,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,88,79,13,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,88,79,13,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,88,79,13,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
traindf = np.array(train_cats)
testdf = np.array(test_cats)

xgb_preds = []

In [26]:
K = 10
kf = KFold(n_splits = K, random_state = 42, shuffle = True)

In [27]:
start = time()

for train_index, test_index in kf.split(traindf):
    train_X, valid_X = traindf[train_index], traindf[test_index]
    train_y, valid_y = target_train[train_index], target_train[test_index]

    # params configuration also from the1owl's kernel
    # https://www.kaggle.com/the1owl/forza-baseline
    xgb_params = {'eta': 0.02, 'max_depth': 8, 'subsample': 0.9, 'colsample_bytree': 0.9, 
                  'objective': 'binary:logistic', 'eval_metric': 'auc', 'seed': 99, 'silent': True}

    d_train = xgb.DMatrix(train_X, train_y)
    d_valid = xgb.DMatrix(valid_X, valid_y)
    d_test = xgb.DMatrix(testdf)
    
    watchlist = [(d_train, 'train'), (d_valid, 'valid')]
    model = xgb.train(xgb_params, d_train, 5000,  watchlist, 
                      maximize=True, verbose_eval=50, early_stopping_rounds=50)
                        
    xgb_pred = model.predict(d_test)
    xgb_preds.append(list(xgb_pred))

end = time()
print ('Time taken is:', end-start)

[0]	train-auc:0.556852	valid-auc:0.560384
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 50 rounds.
[50]	train-auc:0.587965	valid-auc:0.590309
[100]	train-auc:0.589824	valid-auc:0.592988
[150]	train-auc:0.590598	valid-auc:0.593284
Stopping. Best iteration:
[131]	train-auc:0.590555	valid-auc:0.593369

[0]	train-auc:0.561205	valid-auc:0.569208
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 50 rounds.
[50]	train-auc:0.586892	valid-auc:0.596156
[100]	train-auc:0.589251	valid-auc:0.598182
[150]	train-auc:0.590578	valid-auc:0.596753
Stopping. Best iteration:
[122]	train-auc:0.589794	valid-auc:0.598491

[0]	train-auc:0.561409	valid-auc:0.56721
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 50 rounds.
[50]	train-auc:0.590539	valid-auc:0.579252

In [28]:
preds=[]
for i in range(len(xgb_preds[0])):
    sum=0
    for j in range(K):
        sum+=xgb_preds[j][i]
    preds.append(sum / K)

out = pd.DataFrame({'id': id_test, 'is_click': preds})
out.to_csv("pred2.csv", index=False)